In [1]:
import xgboost as xgb
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
filepath = "D:\SFU\\733-Lab\project\IoTScenarios\iot23_combined.csv"

In [3]:
df = pd.read_csv(filepath)

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.0,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0.0,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0.0,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0.0,0,0,0.0,1.0,40.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0.0,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4799995,0.0,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4799996,0.0,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4799997,0.0,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4799998,0.0,0,0,0.0,0.0,0.0,0.0,0.0,DDoS,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df['label'].value_counts()

PartOfAHorizontalPortScan    1200000
Okiru                        1200000
Benign                       1200000
DDoS                         1200000
Name: label, dtype: int64

In [12]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']]
Y = df['label']

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
Y = label_encoder.transform(Y)


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [13]:
num_round = 10
bst = XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=num_round, 
                    objective='binary:logistic')

In [14]:
start = time.time()
print('program start...')
print()

bst.fit(X_train, Y_train)
print()

train_preds = bst.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_preds)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
print()

preds = bst.predict(X_test)
test_accuracy = accuracy_score(Y_test, preds)
print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))

end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...


Train Accuary: 75.49%

Test Accuracy: 75.44%
program end...

time cost: 
55.07123923301697 seconds


In [16]:
import torch
torch.save(bst, 'xgboost.pt')